In [3]:
# last modified 20161108_1600

# added transcripts to table

### Objectives:  
1. topic analysis on transcripts
2. topic analysis on comments
3. video recommender based on common user

In [3]:
# get rid of non-words

In [4]:
# people's names

In [5]:
# get rid of non-english words

In [6]:
# find a way to incorporate pre-processing methods into sklearn tfidf vectoriser

### Imports

In [132]:
import pandas as pd
import numpy as np

import random
import re
from collections import Counter
import time

from pymongo import MongoClient

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from spacy.en import English
parser = English()
import nltk

In [1]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

### Get data from mongoDB

In [24]:
client = MongoClient()
client.database_names()

['fletcher_db', 'local']

In [25]:
transcripts = client.fletcher_db.ts_collection
df_ts = pd.DataFrame(list(transcripts.find()))
df_ts.shape

(2306, 3)

In [26]:
df_ts.columns

Index(['_id', 'transcript', 'url'], dtype='object')

In [124]:
ts_list = [i for i in df_ts.transcript]
len(ts_list)
sample = ts_list[1]

### Transcripts - pre-processing

In [108]:
names = nltk.corpus.names
male_names = names.words('male.txt')
female_names = names.words('female.txt')
male_names = [w.lower() for w in male_names]
male_names_plur = [(w.lower() + "s") for w in male_names]
female_names_plur = [(w.lower() + "s") for w in female_names]
female_names = [w.lower() for w in female_names]
# casenames = list(pd.read_csv("casetitles.csv",encoding = 'iso-8859-1'))
# statenames = list(pd.read_csv("statenames.csv"))

In [109]:
selected_words = ['',
                  'laughter',
                  'applause']

In [134]:
stoppers = set(stopwords.words('english') + list(selected_words) + list(ENGLISH_STOP_WORDS) +
               list(female_names) + list(male_names) + list(female_names_plur) + list(male_names_plur))
stoppers = list(stoppers)

In [135]:
import csv

with open('stoppers.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow([stoppers])

In [136]:
@timefunc
def tokenize(sample):
    separators = ["\xa0\xa0\xa0\xa0", "\r", "\n", "\t", "n't", "'m", "'ll", '[^a-z ]']
    for i in separators:
        sample = re.sub(i, " ", sample.lower())
    
    ## get the tokens using spaCy - this makes it possible to lemmatize the words
    tokens = parser(sample)
    tokens = [tok.lemma_.strip() for tok in tokens]

    ## apply our stoplist
    return [tok for tok in tokens if len(tok) != 1 and tok not in stoppers]
    print('hi')

In [114]:
# # Porter stemmer
# from nltk.stem.porter import *

# def stemmer(token_list):
#     stemmer = PorterStemmer()
#     t_l = [stemmer.stem(i) for i in token_list]
#     return t_l

In [137]:
token_sample = tokenize(sample)

tokenize took 1.3897509574890137 seconds


In [138]:
x = Counter(token_sample)
x.most_common()

[('like', 24),
 ('software', 18),
 ('make', 14),
 ('know', 13),
 ('word', 12),
 ('microsoft', 12),
 ('new', 12),
 ('feature', 11),
 ('come', 11),
 ('time', 11),
 ('design', 10),
 ('use', 10),
 ('thing', 10),
 ('ok', 9),
 ('apple', 9),
 ('company', 9),
 ('write', 9),
 ('buy', 8),
 ('year', 8),
 ('people', 8),
 ('type', 7),
 ('good', 7),
 ('want', 7),
 ('tell', 7),
 ('let', 7),
 ('think', 7),
 ('window', 7),
 ('right', 6),
 ('simplicity', 6),
 ('code', 6),
 ('tap', 6),
 ('thank', 6),
 ('easy', 5),
 ('small', 5),
 ('little', 5),
 ('box', 5),
 ('mean', 5),
 ('world', 5),
 ('choose', 5),
 ('day', 5),
 ('menu', 5),
 ('ca', 5),
 ('really', 5),
 ('feel', 5),
 ('email', 5),
 ('add', 5),
 ('version', 5),
 ('screen', 5),
 ('send', 4),
 ('period', 4),
 ('happen', 4),
 ('simple', 4),
 ('start', 4),
 ('squeak', 4),
 ('power', 4),
 ('work', 4),
 ('sound', 4),
 ('document', 4),
 ('believe', 4),
 ('silence', 4),
 ('old', 4),
 ('sell', 4),
 ('computer', 4),
 ('button', 4),
 ('try', 4),
 ('column', 4),
 